In [109]:
import pandas as pd
import csv
import numpy as np

csv_path = r"C:\Users\START\OneDrive\Desktop\selfcode\complicated_V02.csv"

# 1️⃣ Načtení CSV
df = pd.read_csv(
    csv_path,
    sep=",",
    quoting=csv.QUOTE_MINIMAL,
    encoding='utf-8',
    engine='python',
    on_bad_lines='skip'
)

# 2️⃣ Očištění názvů sloupců
df.columns = df.columns.str.strip()

# 3️⃣ Převod sloupců Id, Line Source, Line Destination na integer (tam kde to jde)
def to_int_or_nan(x):
    try:
        if pd.isna(x) or str(x).strip() == "":
            return np.nan
        return int(float(x))
    except:
        return np.nan

for col in ["Id", "Line Source", "Line Destination"]:
    df[col] = df[col].apply(to_int_or_nan)

# 4️⃣ Přidání nových sloupců
df["Next Step"] = pd.NA
df["Conditional Step"] = pd.NA
df["AI Instrukce"] = pd.NA
df["Memory"] = pd.NA
df["Decision"] = pd.NA

# 5️⃣ Najdi spojky (řádky kde Name == "Line")
lines = df[df["Name"] == "Line"]

# 6️⃣ Vyplnění Next Step (vyloučení Internal storage jako cílového objektu)
internal_ids = df[df["Name"] == "Internal storage"]["Id"].dropna().unique()
for _, line in lines.iterrows():
    src = line["Line Source"]
    dst = line["Line Destination"]
    if pd.notna(src) and pd.notna(dst) and dst not in internal_ids:
        mask = (df["Id"] == src)
        df.loc[mask, "Next Step"] = dst

# 7️⃣ Přenos AI Instrukcí z Preparation bloků do řádků podle Next Step
preparations = df[df["Name"] == "Preparation"]
for _, prep in preparations.iterrows():
    text = prep["Text Area 1"]
    next_step_id = prep["Next Step"]
    if pd.notna(next_step_id):
        mask = df["Id"] == next_step_id
        df.loc[mask, "AI Instrukce"] = text

# 8️⃣ Přenos Memory z Internal storage (reverzní směr)
internal_storage = df[df["Name"] == "Internal storage"]
for _, mem_row in internal_storage.iterrows():
    text = mem_row["Text Area 1"]
    target_id = mem_row["Id"]
    if pd.notna(target_id):
        incoming_links = lines[lines["Line Destination"] == target_id]
        for _, link in incoming_links.iterrows():
            src_id = link["Line Source"]
            if pd.notna(src_id):
                df.loc[df["Id"] == src_id, "Memory"] = text

# 9️⃣ Rozšíření Decision bloků pro každý Conditional Step s přidáním Decision textu
new_rows = []
decisions = df[df["Name"] == "Decision"]

for _, decision_row in decisions.iterrows():
    dec_id = decision_row["Id"]
    if pd.isna(dec_id):
        continue
    matching_lines = lines[lines["Line Source"] == dec_id]
    for _, line in matching_lines.iterrows():
        dest = line["Line Destination"]
        if pd.notna(dest):
            new_row = decision_row.copy()
            new_row["Conditional Step"] = dest
            new_row["Decision"] = line["Text Area 1"]
            new_rows.append(new_row)

# 🔟 Spojení původních a nových Decision řádků
df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

# 🔁 Vytvoření RUS Id (z Text Area 1, část před \n)
df["RUS Id"] = df["Text Area 1"].fillna("").apply(lambda x: x.split('\n')[0])

# ✂️ Úprava Text Area 1 → jen text za \n
def after_first_line(text):
    parts = str(text).split('\n', 1)
    return parts[1].strip() if len(parts) > 1 else ""

df["Step"] = df["Text Area 1"].apply(after_first_line)

# 🗺️ Mapování Next Step a Conditional Step do RUS Id (před odstraněním řádků)
id_to_rusid = df.set_index("Id")["RUS Id"].to_dict()
df["Next Step RUS"] = df["Next Step"].map(id_to_rusid).fillna("")
df["Conditional Step RUS"] = df["Conditional Step"].map(id_to_rusid).fillna("")

# 1️⃣1️⃣ Odstranění nepotřebných řádků
df_expanded = df[
    (~df["Name"].isin(["Line", "Document", "Page", "Internal storage", "Preparation"])) &
    ~((df["Name"] == "Decision") & df["Next Step"].isna() & df["Conditional Step"].isna())
].copy()

# 🧼 Úklid a seřazení sloupců
cols_to_drop = [
    "Name", "Shape Library", "Page ID", "Contained By", "Group",
    "Line Source", "Line Destination", "Source Arrow", "Destination Arrow", "Status", "Text Area 1"
]

df_expanded = df_expanded.drop(columns=[col for col in cols_to_drop if col in df_expanded.columns])
df_expanded = df_expanded.sort_values(by="Id")

desired_order = [
    "Id", "RUS Id", "Step", "Memory", "Decision",
    "Next Step RUS", "Conditional Step RUS", "AI Instrukce",
    "Comments", "Next Step", "Conditional Step"
]

remaining_cols = [col for col in df_expanded.columns if col not in desired_order]
new_order = desired_order + remaining_cols
df_expanded = df_expanded[new_order]


df_expanded.tail(20)

C:\Users\START\AppData\Local\Temp\ipykernel_684\3167984426.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)


,Id,RUS Id,Step,Memory,Decision,Next Step RUS,Conditional Step RUS,AI Instrukce,Comments,Next Step,Conditional Step,Text Area 2,Text Area 3
79,80,PR_,{ostatní negace ze situace dPR22},<NA>,NaN,PR_,,<NA>,NaN,73.0,NaN,NaN,NaN
269,83,PR8,"""Měl/a jsi tam nějaké nepříjemné pocity""",<NA>,Ne,PR10,PR9,<NA>,NaN,47.0,84.0,NaN,NaN
270,83,PR8,"""Měl/a jsi tam nějaké nepříjemné pocity""",<NA>,Ano,PR10,PR10,<NA>,NaN,47.0,47.0,NaN,NaN
82,83,PR8,"""Měl/a jsi tam nějaké nepříjemné pocity""",<NA>,NaN,PR10,,<NA>,NaN,47.0,NaN,NaN,NaN
83,84,PR9,"""Takovou událost nehledáme. Najdi jinou, kde j...",<NA>,NaN,PR1; PR2;PR3;PR4;PR5,,<NA>,NaN,45.0,NaN,NaN,NaN
84,85,PR12,"""Jak bys to řekl/a jinak """,<NA>,NaN,PR11,,<NA>,NaN,48.0,NaN,NaN,NaN
86,87,PR15,"""Takže jsi vždycky{\n0. negace dPR10\nx. negac...",<NA>,NaN,PR16,,<NA>,NaN,89.0,NaN,NaN,NaN
271,87,PR15,"""Takže jsi vždycky{\n0. negace dPR10\nx. negac...",<NA>,Ne,PR16,PR18,<NA>,NaN,89.0,88.0,NaN,NaN
272,87,PR15,"""Takže jsi vždycky{\n0. negace dPR10\nx. negac...",<NA>,Ano,PR16,PR16,<NA>,NaN,89.0,89.0,NaN,NaN
87,88,PR18,"""Takže kdo to dělal?"" / ""Při čem jsi to zažil?""",<NA>,NaN,PR11,,<NA>,NaN,48.0,NaN,NaN,NaN


In [ ]:
# 💾 Uložení výsledku do CSV
output_path = r"C:\Users\START\OneDrive\Desktop\selfcode\processed_output.csv"
df_expanded.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ Hotovo. Výstup uložen do:", output_path)